In [6]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df= pd.read_excel('datatest.xlsx',header=0)
row_name=df.columns.values
df.insert(2,'sewing',0,True)
df.insert(3,'wating',df['headcount'],True)


In [7]:
df

,sequence,operation,sewing,wating,SAM,headcount,WIP
0,1,Bind Panel,0,4,3.3,4,20
1,2,Over Edge,0,4,3.6,4,25
2,3,Seam Panel,0,20,15.0,20,18
3,4,Seam Crotch,0,7,7.4,7,10
4,5,Hem Legs,0,8,8.6,8,6
5,6,Sew Band,0,9,9.2,9,0


In [8]:
def update_sequence(point_check):
    sequence=point_check[1]
    sew_collect=point_check[2]
    df['sewing'][sequence-1]-=sew_collect
    df['wating'][sequence-1]+=sew_collect
    df['WIP'][sequence-1]+=sew_collect
    
def cal_time_check(time,num_seq,sew,list_point_check):
    output=[]
    point_time=time + df['SAM'][num_seq]
    output.append(point_time)
    output.append(num_seq+1)
    output.append(sew)
    list_point_check.append(output)
    list_point_check.sort()
    
def cal_wip(num_seq,time,list_point_check):
    # num_seq=i ~ row=i-1
    if df['wating'][num_seq] !=0 and df['WIP'][num_seq-1] >0 and df['WIP'][num_seq-1] <= df['wating'][num_seq]:
        cal_time_check(time,num_seq,df['WIP'][num_seq-1],list_point_check)
        df['wating'][num_seq]-=df['WIP'][num_seq-1]
        df['sewing'][num_seq]+=df['WIP'][num_seq-1]
        df['WIP'][num_seq-1]=0
        
    elif df['wating'][num_seq] !=0 and df['WIP'][num_seq-1] > df['wating'][num_seq]:
        cal_time_check(time,num_seq,df['wating'][num_seq],list_point_check)
        df['WIP'][num_seq-1]-=df['wating'][num_seq]
        df['wating'][num_seq]=0
        df['sewing'][num_seq]=df['headcount'][num_seq]
def sequence_firt(time,list_point_check):
    df['wating'][0]=0
    df['sewing'][0]=4
    if (time%df['SAM'][0]==0):
        cal_time_check(time,0,4,list_point_check)
    if (time%df['SAM'][0]==0 and time >0):
        df['WIP'][0]+=4


if __name__=="__main__":
    list_check=[]
    time_step=0
    time_line=100
    for i in range(1,df.shape[0]):
        cal_wip(i,time_step,list_check)
    sequence_firt(time_step,list_check)
    pcheck=list_check.pop(0)
    time_step=pcheck[0]
    while time_step<= time_line:
        update_sequence(pcheck)
        for i in range(1,df.shape[0]):
            cal_wip(i,time_step,list_check)
        sequence_firt(time_step,list_check)
        # print(time_step)
        # print(df)
        try:
            pcheck=list_check.pop(0)
            time_step=pcheck[0]
        except:
            break
print(df)

C:\Users\tanguy12\AppData\Local\Temp\ipykernel_10760\2900611830.py:27: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['WIP'][num_seq-1]-=df['wating'][num_seq]
C:\Users\tanguy12\AppData\Local\Temp\ipykernel_10760\2900611830.py:27: SettingWi

   sequence    operation  sewing  wating   SAM  headcount  WIP
0         1   Bind Panel       4       0   3.3          4    0
1         2    Over Edge       0       4   3.6          4    0
2         3   Seam Panel       0      20  15.0         20    1
3         4  Seam Crotch       7       0   7.4          7    5
4         5     Hem Legs       8       0   8.6          8    0
5         6     Sew Band       8       1   9.2          9   86


C:\Users\tanguy12\AppData\Local\Temp\ipykernel_10760\2900611830.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sewing'][sequence-1]-=sew_collect
C:\Users\tanguy12\AppData\Local\Temp\ipykernel_10760\2900611830.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assig